In [189]:
import cv2
import numpy as np
from PIL import Image
import os
import random
import json
import ast
import csv

In [2]:
all_files = os.listdir("D:\\Project Sherlock\\Grafiki")

In [3]:
all_files

['archaive', 'Grafiki total']

In [4]:
colors_dict_hex = {"Red":"#C51111",
                    "Blue": "#132ed1",
                   "Green": "#117f2d",
                   "Pink": "#ed54ba",
                   "White": "#d6e0f0",
                   "Black": "#3f474e",
                   "Yellow": "#f5f557",
                   "Orange": "#ef7d0d",
                   "Purple": "#6b2fbb",
                   "Brown": "#71491e",
                   "Cyan": "#38fedc",
                   "Lime": "#50ef39",
                    "Gray":"",
                      "Banana":"",
                  "Rose":"",
                  "Maroon":"",
                  "Tan":"",
                  "Coral":""}


In [5]:
colors_dict_hex.keys()

dict_keys(['Red', 'Blue', 'Green', 'Pink', 'White', 'Black', 'Yellow', 'Orange', 'Purple', 'Brown', 'Cyan', 'Lime', 'Gray', 'Banana', 'Rose', 'Maroon', 'Tan', 'Coral'])

In [6]:
names = ['Red', 'Blue', 'Green', 'Pink', 'White', 'Black', 'Yellow', 'Orange', 
         'Purple', 'Brown', 'Cyan', 'Lime', 'Gray', 'Banana', 'Rose', 'Maroon', 
         'Tan', 'Coral']


### 2.1 Sprity gracza

In [ ]:
for x in range(len(all_files)):
    # Wczytaj obraz
    img = cv2.imread("D:\\Project Sherlock\\Grafiki\\PC\\{}".format(all_files[x]))
    

    # Koordynaty (przykład)
    x1, y1 = 1800, 540
    x2, y2 = 2170, 1030

    # Wycinamy fragment
    cropped = img[y1:y2, x1:x2]

    # Szukamy czarnej obwódki — tworzymy maskę "ciemnych" pikseli
    # Zakładamy, że obwódka ma wartości RGB bliskie 0
    lower_black = np.array([0, 0, 0])
    upper_black = np.array([50, 50, 50])
    mask_black = cv2.inRange(cropped, lower_black, upper_black)

    # Znalezienie największego konturu (cała postać z obwódką)
    contours, _ = cv2.findContours(mask_black, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)

        # Tworzymy pustą maskę
        mask = np.zeros(cropped.shape[:2], dtype=np.uint8)

        # Wypełniamy kontur (wnętrze postaci + obwódka) na biało
        cv2.drawContours(mask, [largest_contour], -1, 255, thickness=cv2.FILLED)

        # Konwersja maski do kanału alfa
        alpha = mask

        # Rozdziel kanały
        b, g, r = cv2.split(cropped)

        # Łączenie w obraz z alfa
        rgba = cv2.merge([b, g, r, alpha])

        # Zapisz
        cv2.imwrite("D:\\Project Sherlock\\Grafiki\\PC\\{}_masked.png".format(names[x]), rgba)
        print("Zapisano jako {}_masked.png".format(names[x]))
    else:
        print("Nie znaleziono konturu obwódki.")


### 2.2 Czapeczki gracza

In [ ]:
hats_files = os.listdir("D:\\Project Sherlock\\Grafiki\\Hats")[:-1]

In [ ]:


# Parametry pozycji i rozmiaru kwadratów
start_x, start_y = 60, 380
end_x, end_y = 260, 580

# Odstępy między kwadratami
gap_x = 22
gap_y = 38

# Wymiary jednego kwadratu
square_width = end_x - start_x
square_height = end_y - start_y

# Wczytaj obraz


bg_color = np.array([145, 181, 181])

# Liczba kolumn i wierszy (dopasuj do tego, co masz na screenie)
num_cols = 4
num_rows = 3


tolerance = 30

# Liczba pikseli do odcięcia z każdej strony
crop_margin = 24

counter = 0

for file in hats_files:
    img = cv2.imread("D:\\Project Sherlock\\Grafiki\\Hats\\{}".format(file))
    for row in range(num_rows):
        for col in range(num_cols):
            x1 = start_x + col * (square_width + gap_x)
            y1 = start_y + row * (square_height + gap_y)
            x2 = x1 + square_width
            y2 = y1 + square_height

            cropped = img[y1:y2, x1:x2].copy()

            # Oblicz różnicę koloru pikseli od koloru tła
            diff = cv2.absdiff(cropped, bg_color)
            mask = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)

            # Utwórz maskę binarną — piksele podobne do tła będą czarne (0), inne białe (255)
            _, alpha = cv2.threshold(mask, tolerance, 255, cv2.THRESH_BINARY_INV)
            alpha = cv2.bitwise_not(alpha)

            # Rozdziel kanały
            b, g, r = cv2.split(cropped)

            # Połącz z kanałem alfa
            rgba = cv2.merge([b, g, r, alpha])

            h, w = rgba.shape[:2]
            cropped_final = rgba[crop_margin:h-crop_margin, crop_margin:w-crop_margin]

            output_path = f"D:\\Project Sherlock\\Grafiki\\Hats\\icons_output\icon_{counter:02d}.png"
            cv2.imwrite(output_path, cropped_final)
            print(f"Zapisano: {output_path}")

            counter += 1

print("Gotowe! Wszystkie ikony wycięte.")


#### 2.2.2 Hats from the store

In [49]:
# creating a folder

In [54]:
hats_files = os.listdir("D:\\Project Sherlock\\Grafiki\\V2\\Hats")[:-1]

In [55]:
hats_files

['20250708210942_1.jpg',
 '20250708210945_1.jpg',
 '20250708210948_1.jpg',
 '20250708210950_1.jpg',
 '20250708210952_1.jpg',
 '20250708210954_1.jpg',
 '20250708210957_1.jpg',
 '20250708210958_1.jpg',
 '20250708211000_1.jpg',
 '20250708211002_1.jpg',
 '20250708211004_1.jpg']

In [72]:

# Parametry pozycji i rozmiaru kwadratów
start_x, start_y = 75, 200
end_x, end_y = 360, 600

gap_x = 120
gap_y = 100

square_width = end_x - start_x
square_height = end_y - start_y

bg_color = np.array([145, 181, 181])
tolerance = 30
crop_margin = 24

num_cols = 6
num_rows = 2

counter = 0
hats_files = os.listdir("D:\\Project Sherlock\\Grafiki\\V2\\Hats")[:-1]

for file in hats_files:
    img = cv2.imread(f"D:\\Project Sherlock\\Grafiki\\V2\\Hats\\{file}")

    for row in range(num_rows):
        for col in range(num_cols):
            x1 = start_x + col * (square_width + gap_x)
            y1 = start_y + row * (square_height + gap_y)
            x2 = x1 + square_width
            y2 = y1 + square_height

            # Wytnij kwadrat i usuń dolne 60 px
            cropped = img[(y1+60):y2, x1:x2].copy()

            # Oblicz różnicę koloru pikseli od koloru tła
            diff = cv2.absdiff(cropped, bg_color)
            mask = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)

            # Utwórz maskę binarną — piksele podobne do tła będą czarne (0), inne białe (255)
            _, alpha = cv2.threshold(mask, tolerance, 255, cv2.THRESH_BINARY_INV)
            alpha = cv2.bitwise_not(alpha)

            # Rozdziel kanały
            b, g, r = cv2.split(cropped)
            rgba = cv2.merge([b, g, r, alpha])

            # Odetnij margines
            h, w = rgba.shape[:2]
            cropped_final = rgba[crop_margin:h-crop_margin, crop_margin:w-crop_margin]

            # Sprawdź kanały — jeśli brak alfa, dodaj
            if cropped_final.shape[2] == 3:
                b, g, r = cv2.split(cropped_final)
                alpha = np.ones(b.shape, dtype=np.uint8) * 255
                cropped_final = cv2.merge([b, g, r, alpha])

            # Konwersja do szarości, by wykryć obwódkę
            gray = cv2.cvtColor(cropped_final[:, :, :3], cv2.COLOR_BGR2GRAY)
            _, thresh = cv2.threshold(gray, 40, 255, cv2.THRESH_BINARY_INV)

            # Szukanie największego konturu
            contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            if contours:
                largest_contour = max(contours, key=cv2.contourArea)

                # Maska wypełniająca kontur
                mask = np.zeros_like(gray)
                cv2.drawContours(mask, [largest_contour], -1, 255, thickness=cv2.FILLED)

                # Nowy kanał alfa
                new_alpha = mask

                # Wyodrębnij kanały z poprawionego cropped_final
                b, g, r, a = cv2.split(cropped_final)

                # Scal nowy obraz
                final_img = cv2.merge([b, g, r, new_alpha])

                # Bounding box, żeby przyciąć dokładniej
                x, y, w_box, h_box = cv2.boundingRect(largest_contour)
                final_cropped = final_img[y:y+h_box, x:x+w_box]
            else:
                # Jeśli nie ma konturu, użyj samego cropped_final
                final_cropped = cropped_final

            # Zapisz wynik
            output_path = f"D:\\Project Sherlock\\Grafiki\\V2\\Hats\\icons_output\\icon_{counter:02d}.png"
            cv2.imwrite(output_path, final_cropped)
            counter += 1

print("Gotowe! Wszystkie ikony wycięte.")


Gotowe! Wszystkie ikony wycięte.


### 2.3 Google Gracza

In [ ]:
eye_files = os.listdir("D:\\Project Sherlock\\Grafiki\\Hats")[:-1]

In [ ]:

# Parametry pozycji i rozmiaru kwadratów
start_x, start_y = 60, 380
end_x, end_y = 260, 580

# Odstępy między kwadratami
gap_x = 22
gap_y = 38

# Wymiary jednego kwadratu
square_width = end_x - start_x
square_height = end_y - start_y

# Wczytaj obraz


bg_color = np.array([145, 181, 181])

# Liczba kolumn i wierszy (dopasuj do tego, co masz na screenie)
num_cols = 4
num_rows = 3


tolerance = 30

# Liczba pikseli do odcięcia z każdej strony
crop_margin = 24

counter = 0

for file in eye_files:
    img = cv2.imread("D:\\Project Sherlock\\Grafiki\\Eyes\\{}".format(file))
    for row in range(num_rows):
        for col in range(num_cols):
            x1 = start_x + col * (square_width + gap_x)
            y1 = start_y + row * (square_height + gap_y)
            x2 = x1 + square_width
            y2 = y1 + square_height

            cropped = img[y1:y2, x1:x2].copy()

            # Oblicz różnicę koloru pikseli od koloru tła
            diff = cv2.absdiff(cropped, bg_color)
            mask = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)

            # Utwórz maskę binarną — piksele podobne do tła będą czarne (0), inne białe (255)
            _, alpha = cv2.threshold(mask, tolerance, 255, cv2.THRESH_BINARY_INV)
            alpha = cv2.bitwise_not(alpha)

            # Rozdziel kanały
            b, g, r = cv2.split(cropped)

            # Połącz z kanałem alfa
            rgba = cv2.merge([b, g, r, alpha])

            h, w = rgba.shape[:2]
            cropped_final = rgba[crop_margin:h-crop_margin, crop_margin:w-crop_margin]

            output_path = f"D:\\Project Sherlock\\Grafiki\\Eyes\\icons_output\icon_{counter:02d}.png"
            cv2.imwrite(output_path, cropped_final)
            print(f"Zapisano: {output_path}")

            counter += 1

print("Gotowe! Wszystkie ikony wycięte.")


In [ ]:
n_list_names = []
n_list_names_2 = []
n_start = 93
for x in range(1,len(os.listdir("D:\\Project Sherlock\\Grafiki\\Grafiki total\\Hats\\2"))):
    new_name = "icon_{}.jpg".format(str(n_start + x))
    n_list_names_2.append(new_name)
    new_name = "icon_{}".format(str(n_start + x))
    n_list_names.append(new_name)
    
    
old_list_names = os.listdir("D:\\Project Sherlock\\Grafiki\\Grafiki total\\Hats\\2")
path_folder = "D:\\Project Sherlock\\Grafiki\\Grafiki total\\Hats\\2"

for x in range(0, len(os.listdir(path_folder))):
    os.rename("{}\\{}".format(path_folder,n_list_names[x]), "{}\\{}".format(path_folder,n_list_names_2[x]))


#### 2.3.2 Eyes from shoop

In [25]:
# Parametry pozycji i rozmiaru kwadratów
start_x, start_y = 75, 200
end_x, end_y = 360, 600

gap_x = 120
gap_y = 100

square_width = end_x - start_x
square_height = end_y - start_y

bg_color = np.array([145, 181, 181])
tolerance = 30
crop_margin = 24

num_cols = 6
num_rows = 3

counter = 5
hats_files = os.listdir("D:\\Project Sherlock\\Grafiki\\V2\\Eyes")[:-1]

for file in hats_files:
    img = cv2.imread(f"D:\\Project Sherlock\\Grafiki\\V2\\Eyes\\{file}")

    for row in range(num_rows):
        for col in range(num_cols):
            x1 = start_x + col * (square_width + gap_x)
            y1 = start_y + row * (square_height + gap_y)
            x2 = x1 + square_width
            y2 = y1 + square_height

            # Wytnij kwadrat i usuń dolne 60 px
            cropped = img[(y1):y2, x1:x2].copy()

            # Oblicz różnicę koloru pikseli od koloru tła
            diff = cv2.absdiff(cropped, bg_color)
            mask = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)

            # Utwórz maskę binarną — piksele podobne do tła będą czarne (0), inne białe (255)
            _, alpha = cv2.threshold(mask, tolerance, 255, cv2.THRESH_BINARY_INV)
            alpha = cv2.bitwise_not(alpha)

            # Rozdziel kanały
            b, g, r = cv2.split(cropped)
            rgba = cv2.merge([b, g, r, alpha])

            # Odetnij margines
            h, w = rgba.shape[:2]
            cropped_final = rgba[crop_margin:h-crop_margin, crop_margin:w-crop_margin]
      
            # Sprawdź kanały — jeśli brak alfa, dodaj
            if cropped_final.shape[2] == 3:
                b, g, r = cv2.split(cropped_final)
                alpha = np.ones(b.shape, dtype=np.uint8) * 255
                cropped_final = cv2.merge([b, g, r, alpha])

            # Konwersja do szarości, by wykryć obwódkę
            gray = cv2.cvtColor(cropped_final[:, :, :3], cv2.COLOR_BGR2GRAY)
            _, thresh = cv2.threshold(gray, 40, 255, cv2.THRESH_BINARY_INV)

            # Szukanie największego konturu
            contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            if contours:
                largest_contour = max(contours, key=cv2.contourArea)

                # Maska wypełniająca kontur
                mask = np.zeros_like(gray)
                cv2.drawContours(mask, [largest_contour], -1, 255, thickness=cv2.FILLED)

                # Nowy kanał alfa
                new_alpha = mask

                # Wyodrębnij kanały z poprawionego cropped_final
                b, g, r, a = cv2.split(cropped_final)

                # Scal nowy obraz
                final_img = cv2.merge([b, g, r, new_alpha])

                # Bounding box, żeby przyciąć dokładniej
                x, y, w_box, h_box = cv2.boundingRect(largest_contour)
                final_cropped = final_img[y:y+h_box, x:x+w_box]
            else:
                # Jeśli nie ma konturu, użyj samego cropped_final
                final_cropped = cropped_final
    
            # Zapisz wynik
            output_path = f"D:\\Project Sherlock\\Grafiki\\V2\\Eyes\\icons_output\\icon_{counter:02d}.png"
            cv2.imwrite(output_path, final_cropped)
            counter += 1

print("Gotowe! Wszystkie ikony wycięte.")


Gotowe! Wszystkie ikony wycięte.


### 2.4 Ubranko Gracza

In [2]:
pants_files = os.listdir("D:\\Project Sherlock\\Grafiki\\Pants")[:-1]

In [3]:
pants_files

['archive',
 'icon_01.png',
 'icon_02.png',
 'icon_03.png',
 'icon_04.png',
 'icon_05.png',
 'icon_06.png']

In [34]:
# Parametry pozycji i rozmiaru kwadratów
start_x, start_y = 55, 480
end_x, end_y = 370, 860

gap_x = 120
gap_y = 100

square_width = end_x - start_x
square_height = end_y - start_y

bg_color = np.array([145, 181, 181])
tolerance = 30
crop_margin = 24

num_cols = 6
num_rows = 2

counter = 5
hats_files = os.listdir("D:\\Project Sherlock\\Grafiki\\V2\\Pants")[:-1]

for file in hats_files:
    img = cv2.imread(f"D:\\Project Sherlock\\Grafiki\\V2\\Pants\\{file}")
    print(file)
    for row in range(num_rows):
        for col in range(num_cols):
            x1 = start_x + col * (square_width + gap_x)
            y1 = start_y + row * (square_height + gap_y)
            x2 = x1 + square_width
            y2 = y1 + square_height

            # Wytnij kwadrat i usuń dolne 60 px
            cropped = img[(y1):y2, x1:x2].copy()

            # Oblicz różnicę koloru pikseli od koloru tła
            bg = np.full(cropped.shape, bg_color, dtype=np.uint8)
            diff = cv2.absdiff(cropped, bg)
            mask = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)

            # Utwórz maskę binarną — piksele podobne do tła będą czarne (0), inne białe (255)
            _, alpha = cv2.threshold(mask, tolerance, 255, cv2.THRESH_BINARY_INV)
            alpha = cv2.bitwise_not(alpha)

            # Rozdziel kanały
            b, g, r = cv2.split(cropped)
            rgba = cv2.merge([b, g, r, alpha])

            # Odetnij margines
            h, w = rgba.shape[:2]
            cropped_final = rgba[crop_margin:h-crop_margin, crop_margin:w-crop_margin]
      
            # Sprawdź kanały — jeśli brak alfa, dodaj
            if cropped_final.shape[2] == 3:
                b, g, r = cv2.split(cropped_final)
                alpha = np.ones(b.shape, dtype=np.uint8) * 255
                cropped_final = cv2.merge([b, g, r, alpha])

            # Konwersja do szarości, by wykryć obwódkę
            gray = cv2.cvtColor(cropped_final[:, :, :3], cv2.COLOR_BGR2GRAY)
            _, thresh = cv2.threshold(gray, 40, 255, cv2.THRESH_BINARY_INV)

            # Szukanie największego konturu
            contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            if contours:
                largest_contour = max(contours, key=cv2.contourArea)

                # Maska wypełniająca kontur
                mask = np.zeros_like(gray)
                cv2.drawContours(mask, [largest_contour], -1, 255, thickness=cv2.FILLED)

                # Nowy kanał alfa
                new_alpha = mask

                # Wyodrębnij kanały z poprawionego cropped_final
                b, g, r, a = cv2.split(cropped_final)

                # Scal nowy obraz
                final_img = cv2.merge([b, g, r, new_alpha])

                # Bounding box, żeby przyciąć dokładniej
                x, y, w_box, h_box = cv2.boundingRect(largest_contour)
                final_cropped = final_img[y:y+h_box, x:x+w_box]
            else:
                # Jeśli nie ma konturu, użyj samego cropped_final
                final_cropped = cropped_final
    
            # Zapisz wynik
            output_path = f"D:\\Project Sherlock\\Grafiki\\V2\\Pants\\icons_output\\icon_{counter:02d}.png"
            cv2.imwrite(output_path, final_cropped)
            counter += 1

print("Gotowe! Wszystkie ikony wycięte.")


20250708211023_1.jpg
20250708211026_1.jpg
Gotowe! Wszystkie ikony wycięte.


### 3. Clean up images

#### 3.1 Rename them from 0 and more

In [12]:
n_list_names = []
n_start = 0
for x in range(0,len(os.listdir("D:\\Project Sherlock\\Grafiki\\Grafiki total\\Pants"))):
    new_name = "icon_{}.jpg".format(str(x+ n_start))
    n_list_names.append(new_name)
    
old_list_names = os.listdir("D:\\Project Sherlock\\Grafiki\\Grafiki total\\Pants")
path_folder = "D:\\Project Sherlock\\Grafiki\\Grafiki total\\Pants"

for x in range(0, len(os.listdir(path_folder))):
    os.rename("{}\\{}".format(path_folder,old_list_names[x]), "{}\\{}".format(path_folder,n_list_names[x]))


#### 3.2 Resize the images to unify them

In [ ]:

# Folder paths
input_folder = "D:\\Project Sherlock\\Grafiki\\Grafiki total\\Pants"   # <- Change this
output_folder = "D:\\Project Sherlock\\Grafiki\\Grafiki total\\Pants\\resized"  # <- Change this

# Desired size
target_size = (152, 152)

# Create output folder if not exist
os.makedirs(output_folder, exist_ok=True)

# Loop through all files
for filename in os.listdir(input_folder):
    if filename.endswith(".png") or filename.endswith(".jpg") or filename.endswith(".jpeg"):
        img_path = os.path.join(input_folder, filename)
        img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)

        if img is not None:
            resized = cv2.resize(img, target_size, interpolation=cv2.INTER_AREA)

            output_path = os.path.join(output_folder, filename).replace(".jpg",".png")
            cv2.imwrite(output_path, resized)
            print(f"Saved: {output_path}")
        else:
            print(f"Could not read: {filename}")

print("✅ Done!")


resized = cv2.resize(img, (152, 152), interpolation=cv2.INTER_AREA)

## 4. Create

###  4.1 Random player

In [229]:
def overlay_image(background, overlay, x, y, scale_factor):
    """Osadza overlay na background w pozycji (x, y) z obsługą alfa."""
    
    
    
    new_w = int(overlay.shape[1] * scale_factor)
    new_h = int(overlay.shape[0] * scale_factor)

    # Użyj cv2.resize
    overlay = cv2.resize(overlay, (new_w, new_h), interpolation=cv2.INTER_LINEAR)


    bh, bw = background.shape[:2]
    oh, ow = overlay.shape[:2]

    if x + ow > bw or y + oh > bh:
        print("Ostrzeżenie: overlay wykracza poza obraz tła!")
       # print(x+ow, y+oh)
        return background

    # Wyodrębnij kanały
    b, g, r, a = cv2.split(overlay)
    mask = a / 255.0

    for c in range(3):  # RGB
        background[y:y+oh, x:x+ow, c] = (1.0 - mask) * background[y:y+oh, x:x+ow, c] + mask * overlay[:, :, c]

    # Nadpisujemy kanał alfa tła, jeśli potrzebne
    if background.shape[2] == 4:
        background[y:y+oh, x:x+ow, 3] = np.maximum(background[y:y+oh, x:x+ow, 3], a)

    return background

In [117]:
def create_random_crewmate(hat_path, eye_path, pant_path, pc_path, path, out_name):
    # Wczytaj obraz bazowy
    base_img = cv2.imread(pc_path, cv2.IMREAD_UNCHANGED)
    h, w = base_img.shape[:2]
    
    # Nowe wymiary
    new_w = w + 70 + 70
    new_h = h + 160 +40

    # Tworzymy nowy obraz (przezroczysty)
    new_img = np.zeros((new_h, new_w, 4), dtype=np.uint8)

    # Wklejamy oryginał na odpowiednie miejsce
    new_img[160:160+h, 50:50+w] = base_img


    # Wczytaj elementy
    hat = cv2.imread(hat_path, cv2.IMREAD_UNCHANGED)
    mask = cv2.imread(mask_path, cv2.IMREAD_UNCHANGED)
    suit = cv2.imread(suit_path, cv2.IMREAD_UNCHANGED)

    # Osadź elementy
    extended_img = overlay_image(new_img, hat, 140, 60, scale_factor = 1.5)
    extended_img = overlay_image(new_img, mask, 200, 180, scale_factor = 1.3)
    extended_img = overlay_image(new_img, suit, 98, 355, scale_factor = 2.1)

    # Zapisz wynik
    cv2.imwrite("{}\\{}".format(path, out_name), new_img)
    print("Zapisano: sprite_with_items.png")

In [148]:
sprites_pc = os.listdir("D:\\Project Sherlock\\Grafiki\\Grafiki total\\PC")[1:]
sprites_eyes = os.listdir("D:\\Project Sherlock\\Grafiki\\Grafiki total\\Eyes")[1:]
sprites_pants = os.listdir("D:\\Project Sherlock\\Grafiki\\Grafiki total\\Pants")[1:]
sprites_hats = os.listdir("D:\\Project Sherlock\\Grafiki\\Grafiki total\\Hats")[1:]
r_pc = random.choice(sprites_pc)
r_hat = random.choice(sprites_hats)
#r_eye = random.choice(sprites_eyes)

r_eye =sprites_eyes[0]
r_pant = random.choice(sprites_pants)


# Plik główny (sprite)
base_img_path = "{}\\PC\\{}".format(save_path,  r_pc)



# Dodatkowe elementy
hat_path = "{}\\Hats\\{}".format(save_path, r_hat)
mask_path = "{}\\Eyes\\{}".format(save_path,r_eye)
suit_path = "{}\\Pants\\{}".format(save_path,r_pant)


save_name = "test_1.png"

create_random_crewmate(hat_path, mask_path, suit_path, base_img_path, save_path, save_name)

Zapisano: sprite_with_items.png


### 4.2 Create coordinates for map - SKELD

In [210]:

# Ścieżki
map_path = "D:\\Project Sherlock\\Grafiki\\Grafiki total\\Maps\\The_Skeld_map.jpg"

char_path = "D:\\Project Sherlock\\Grafiki\\Grafiki total\\test_1.jpg"
char_path = "character.png"

# Zadeklaruj rozmiar okna monitora (przykład: full HD)
monitor_width = 1920
monitor_height = 1080


# Globalne zmienne
drawing = False
ix, iy = -1, -1
spawn_areas = []

# Callback myszki
def draw_rectangle(event, x, y, flags, param):
    global ix, iy, drawing, map_copy, spawn_areas

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        ix, iy = x, y

    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            temp = map_copy.copy()
            # Pokazujemy wszystkie już zapisane prostokąty
            for area in spawn_areas:
                cv2.rectangle(temp, (area["x_min"], area["y_min"]), (area["x_max"], area["y_max"]), (0, 255, 0), 2)
            # Tymczasowy aktualny
            cv2.rectangle(temp, (ix, iy), (x, y), (0, 0, 255), 2)
            cv2.imshow("Map Editor", temp)

    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        x_min = min(ix, x)
        x_max = max(ix, x)
        y_min = min(iy, y)
        y_max = max(iy, y)
        spawn_areas.append({
            "x_min": x_min,
            "x_max": x_max,
            "y_min": y_min,
            "y_max": y_max
        })
        # Aktualizujemy kopię, żeby prostokąt został
        cv2.rectangle(map_copy, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
        print(f"✅ Dodano area: {spawn_areas[-1]}")

# Wczytaj mapę
map_img = cv2.imread(map_path)
map_copy = map_img.copy()


orig_height, orig_width = map_img.shape[:2]

# Oblicz skalę tak, żeby zmieściła się w oknie monitora
scale_w = monitor_width / orig_width
scale_h = monitor_height / orig_height
scale = min(scale_w, scale_h)

new_width = int(orig_width * scale)
new_height = int(orig_height * scale)

# Zmień rozmiar mapy
map_img = cv2.resize(map_img, (new_width, new_height), interpolation=cv2.INTER_AREA)

# Kopia do rysowania
map_copy = map_img.copy()

cv2.namedWindow("Map Editor")
cv2.setMouseCallback("Map Editor", draw_rectangle)

print("▶️ Klikaj i rysuj wiele spawn areas. ESC = zakończ.")

while True:
    cv2.imshow("Map Editor", map_copy)
    k = cv2.waitKey(1) & 0xFF
    if k == 27:  # ESC
        break

cv2.destroyAllWindows()

# Zapisz wszystkie spawn area do JSON
with open("spawn_areas.json", "w") as f:
    json.dump(spawn_areas, f, indent=4)
print("💾 Zapisano spawn_areas.json")

# Wczytaj postać
char_img = cv2.imread(char_path, cv2.IMREAD_UNCHANGED)
h, w = char_img.shape[:2]

# Kopia mapy
result = map_img.copy()

# Dla przykładu: wstawiamy postać do KAŻDEGO area (możesz zmienić np. losowo tylko do jednej)
for area in spawn_areas:
    x_spawn = random.randint(area["x_min"], area["x_max"] - w)
    y_spawn = random.randint(area["y_min"], area["y_max"] - h)

    # Wklej z alfa
    if char_img.shape[2] == 4:
        alpha_s = char_img[:, :, 3] / 255.0
        alpha_l = 1.0 - alpha_s

        for c in range(3):
            result[y_spawn:y_spawn+h, x_spawn:x_spawn+w, c] = (
                alpha_s * char_img[:, :, c] +
                alpha_l * result[y_spawn:y_spawn+h, x_spawn:x_spawn+w, c]
            )
    else:
        result[y_spawn:y_spawn+h, x_spawn:x_spawn+w] = char_img

cv2.imwrite("mapa_with_spawns.png", result)
print("✅ Zapisano: mapa_with_spawns.png")

cv2.imshow("Result", result)
cv2.waitKey(0)
cv2.destroyAllWindows()


▶️ Klikaj i rysuj wiele spawn areas. ESC = zakończ.
✅ Dodano area: {'x_min': 927, 'x_max': 1156, 'y_min': 638, 'y_max': 988}
✅ Dodano area: {'x_min': 915, 'x_max': 1592, 'y_min': 747, 'y_max': 806}
✅ Dodano area: {'x_min': 1408, 'x_max': 1602, 'y_min': 701, 'y_max': 893}
✅ Dodano area: {'x_min': 1467, 'x_max': 1536, 'y_min': 499, 'y_max': 879}
✅ Dodano area: {'x_min': 1562, 'x_max': 1630, 'y_min': 371, 'y_max': 562}
✅ Dodano area: {'x_min': 1569, 'x_max': 1877, 'y_min': 440, 'y_max': 503}
✅ Dodano area: {'x_min': 1743, 'x_max': 1882, 'y_min': 384, 'y_max': 550}
✅ Dodano area: {'x_min': 1318, 'x_max': 1628, 'y_min': 390, 'y_max': 442}
✅ Dodano area: {'x_min': 1475, 'x_max': 1535, 'y_min': 129, 'y_max': 446}
✅ Dodano area: {'x_min': 483, 'x_max': 1532, 'y_min': 218, 'y_max': 265}
✅ Dodano area: {'x_min': 902, 'x_max': 1264, 'y_min': 44, 'y_max': 427}
✅ Dodano area: {'x_min': 1059, 'x_max': 1111, 'y_min': 27, 'y_max': 986}
✅ Dodano area: {'x_min': 1065, 'x_max': 1407, 'y_min': 538, 'y_max

AttributeError: 'NoneType' object has no attribute 'shape'

In [212]:
coordinates_skeld = """{'x_min': 927, 'x_max': 1156, 'y_min': 638, 'y_max': 988}
{'x_min': 915, 'x_max': 1592, 'y_min': 747, 'y_max': 806}
{'x_min': 1408, 'x_max': 1602, 'y_min': 701, 'y_max': 893}
{'x_min': 1467, 'x_max': 1536, 'y_min': 499, 'y_max': 879}
{'x_min': 1562, 'x_max': 1630, 'y_min': 371, 'y_max': 562}
{'x_min': 1569, 'x_max': 1877, 'y_min': 440, 'y_max': 503}
{'x_min': 1743, 'x_max': 1882, 'y_min': 384, 'y_max': 550}
{'x_min': 1318, 'x_max': 1628, 'y_min': 390, 'y_max': 442}
{'x_min': 1475, 'x_max': 1535, 'y_min': 129, 'y_max': 446}
{'x_min': 483, 'x_max': 1532, 'y_min': 218, 'y_max': 265}
{'x_min': 902, 'x_max': 1264, 'y_min': 44, 'y_max': 427}
{'x_min': 1059, 'x_max': 1111, 'y_min': 27, 'y_max': 986}
{'x_min': 1065, 'x_max': 1407, 'y_min': 538, 'y_max': 593}
{'x_min': 1192, 'x_max': 1390, 'y_min': 537, 'y_max': 697}
{'x_min': 670, 'x_max': 853, 'y_min': 305, 'y_max': 515}
{'x_min': 404, 'x_max': 472, 'y_min': 290, 'y_max': 694}
{'x_min': 611, 'x_max': 1161, 'y_min': 829, 'y_max': 892}
{'x_min': 708, 'x_max': 897, 'y_min': 574, 'y_max': 788}
{'x_min': 174, 'x_max': 346, 'y_min': 365, 'y_max': 619}
{'x_min': 182, 'x_max': 638, 'y_min': 469, 'y_max': 534}
{'x_min': 529, 'x_max': 639, 'y_min': 388, 'y_max': 568}
{'x_min': 331, 'x_max': 528, 'y_min': 637, 'y_max': 842}
{'x_min': 326, 'x_max': 519, 'y_min': 153, 'y_max': 343}
{'x_min': 1177, 'x_max': 1386, 'y_min': 832, 'y_max': 990}
{'x_min': 335, 'x_max': 652, 'y_min': 712, 'y_max': 827}
"""

In [213]:
# Usuń białe znaki na początku i końcu
clean_text = coordinates_skeld.strip()

# Podziel na linie
lines = clean_text.splitlines()

# Lista słowników
areas = []

for line in lines:
    line = line.strip()
    if line:
        try:
            area = ast.literal_eval(line)
            areas.append(area)
        except Exception as e:
            print(f"Błąd przy parsowaniu: {line}\n{e}")

# Zapisz do pliku JSON
with open("D:\\Project Sherlock\\spawn_areas.json", "w") as f:
    json.dump(areas, f, indent=4)

print(f"✅ Zapisano {len(areas)} areas do spawn_areas.json")

✅ Zapisano 25 areas do spawn_areas.json


In [214]:
def load_json_file(json_path: str):
    """
    Loads a JSON file and returns its contents.

    Args:
        json_path (str): Path to the JSON file.

    Returns:
        object: Parsed JSON data (e.g., list of dicts).
    """
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    print(f"✅ Wczytano {len(data)} areas z {json_path}")
    return data

In [215]:
areas = load_json_file("D:\\Project Sherlock\\spawn_areas.json")

✅ Wczytano 25 areas z D:\Project Sherlock\spawn_areas.json


In [216]:
areas

[{'x_min': 927, 'x_max': 1156, 'y_min': 638, 'y_max': 988},
 {'x_min': 915, 'x_max': 1592, 'y_min': 747, 'y_max': 806},
 {'x_min': 1408, 'x_max': 1602, 'y_min': 701, 'y_max': 893},
 {'x_min': 1467, 'x_max': 1536, 'y_min': 499, 'y_max': 879},
 {'x_min': 1562, 'x_max': 1630, 'y_min': 371, 'y_max': 562},
 {'x_min': 1569, 'x_max': 1877, 'y_min': 440, 'y_max': 503},
 {'x_min': 1743, 'x_max': 1882, 'y_min': 384, 'y_max': 550},
 {'x_min': 1318, 'x_max': 1628, 'y_min': 390, 'y_max': 442},
 {'x_min': 1475, 'x_max': 1535, 'y_min': 129, 'y_max': 446},
 {'x_min': 483, 'x_max': 1532, 'y_min': 218, 'y_max': 265},
 {'x_min': 902, 'x_max': 1264, 'y_min': 44, 'y_max': 427},
 {'x_min': 1059, 'x_max': 1111, 'y_min': 27, 'y_max': 986},
 {'x_min': 1065, 'x_max': 1407, 'y_min': 538, 'y_max': 593},
 {'x_min': 1192, 'x_max': 1390, 'y_min': 537, 'y_max': 697},
 {'x_min': 670, 'x_max': 853, 'y_min': 305, 'y_max': 515},
 {'x_min': 404, 'x_max': 472, 'y_min': 290, 'y_max': 694},
 {'x_min': 611, 'x_max': 1161, 'y_

### 4.3 Put player on map

In [70]:
def random_coord_in_area_with_size(area):
    """
    Losuje współrzędne w area tak, żeby cały gracz się zmieścił.
    """

    x = random.randint(area["x_min"], area["x_max"])
    y = random.randint(area["y_min"], area["y_max"])
    return (x, y)

In [115]:
def spawn_players_on_map(
        map_path,
        char_path,
        spawn_areas,
        output_image_path,
        output_coords_path,
        num_players=1,
        char_width=80,
        char_height=130,
        scale = 4.4609375
                        ):
    """
    Wstawia graczy na mapę w losowych area.
    Zapisuje nowy obraz oraz plik JSON z koordynatami.
    """

    # Wczytaj mapę
    map_img = cv2.imread(map_path)
    result_img = map_img.copy()

    # Wczytaj postać
    char_img = cv2.imread(char_path, cv2.IMREAD_UNCHANGED)
    if char_img is None:
        raise FileNotFoundError(f"❌ Nie można wczytać postaci: {char_path}")

    # Zmień rozmiar postaci
    char_img_resized = cv2.resize(char_img, (char_width, char_height), interpolation=cv2.INTER_AREA)

    coords_list = []

    for _ in range(num_players):
        
        area = random.choice(spawn_areas)
        coord = random_coord_in_area_with_size(area)
        x, y = coord
        
        x = int(x * scale)
        y = int(y * scale)

        coords_list.append({"x": x, "y": y})
        #coords_list.append({"x": x*scale, "y": y*scale})
        
        # Wklej gracza na mapę
        if char_img_resized.shape[2] == 4:
            # Kanał alfa
            alpha_s = char_img_resized[:, :, 3] / 255.0
            alpha_l = 1.0 - alpha_s

            for c in range(3):
                result_img[y:y+char_height, x:x+char_width, c] = (
                    alpha_s * char_img_resized[:, :, c] +
                    alpha_l * result_img[y:y+char_height, x:x+char_width, c]
                )
        else:
            result_img[y:y+char_height, x:x+char_width] = char_img_resized

    # Zapisz wynikową mapę
    cv2.imwrite(output_image_path, result_img)
    print(f"✅ Zapisano mapę z graczami: {output_image_path}")

    # Zapisz współrzędne
    with open(output_coords_path, "w") as f:
        json.dump(coords_list, f, indent=4)
    print(f"✅ Zapisano koordynaty do: {output_coords_path}")


In [116]:
spawn_players_on_map(
    map_path="D:\\Project Sherlock\\Grafiki\\Grafiki total\\Maps\\The_Skeld_map.jpg",
    char_path="D:\\Project Sherlock\\Grafiki\\Grafiki total\\test_1.png",
    spawn_areas=areas,
    output_image_path="D:\\Project Sherlock\\map_with_players.png",
    output_coords_path="D:\\Project Sherlock\\player_coords.json",
    num_players=1  # ← zmieniasz tu od 1 do 16
)


✅ Zapisano mapę z graczami: D:\Project Sherlock\map_with_players.png
✅ Zapisano koordynaty do: D:\Project Sherlock\player_coords.json


### 4.4 Create screenshots in lopp

In [155]:
areas = load_json_file("D:\\Project Sherlock\\spawn_areas.json")

✅ Wczytano 75 areas z D:\Project Sherlock\spawn_areas.json


In [217]:
def create_random_crewmate_return(sprites_hats, sprites_eyes, sprites_pants, sprites_pc):
    """
    Tworzy obraz postaci z nałożonymi elementami i zwraca go jako numpy array (RGBA).
    
    Args:
        hat_path (str): Ścieżka do pliku czapki.
        mask_path (str): Ścieżka do pliku maski/okularów.
        suit_path (str): Ścieżka do pliku kombinezonu.
        pc_path (str): Ścieżka do bazowej postaci.
    
    Returns:
        np.ndarray: Obraz wynikowy (z kanałem alfa).
    """
    r_pc = random.choice(sprites_pc)
    r_hat = random.choice(sprites_hats)
    r_eye = random.choice(sprites_eyes)
    r_pant = random.choice(sprites_pants)
    
    save_path = "D:\\Project Sherlock\\Grafiki\\Grafiki total"

    # Plik główny (sprite)
    pc_path = "{}\\PC\\{}".format(save_path,  r_pc)
    
    # Dodatkowe elementy
    hat_path = "{}\\Hats\\{}".format(save_path, r_hat)
    mask_path = "{}\\Eyes\\{}".format(save_path,r_eye)
    suit_path = "{}\\Pants\\{}".format(save_path,r_pant)
    # Wczytaj obraz bazowy
    base_img = cv2.imread(pc_path, cv2.IMREAD_UNCHANGED)
    h, w = base_img.shape[:2]
    
    # Nowe wymiary
    new_w = w + 70 + 70
    new_h = h + 160 + 40

    # Tworzymy nowy obraz (przezroczysty)
    new_img = np.zeros((new_h, new_w, 4), dtype=np.uint8)

    # Wklejamy bazowy obraz na nowe płótno
    new_img[160:160+h, 50:50+w] = base_img

    # Wczytaj elementy
    hat = cv2.imread(hat_path, cv2.IMREAD_UNCHANGED)
    mask = cv2.imread(mask_path, cv2.IMREAD_UNCHANGED)
    suit = cv2.imread(suit_path, cv2.IMREAD_UNCHANGED)

    # Osadź elementy
    extended_img = overlay_image(new_img, hat, 140, 60, scale_factor=1.5)
    extended_img = overlay_image(extended_img, mask, 200, 180, scale_factor=1.3)
    extended_img = overlay_image(extended_img, suit, 98, 355, scale_factor=2.1)

    return extended_img

In [230]:
def spawn_players_on_map_instant(
        sprites_hats, 
        sprites_eyes, 
        sprites_pants,
        sprites_pc,
        map_img,
        spawn_areas,
        output_image_path,
        num_players=1,
        char_width=100,
        char_height=150,
        scale = 4.4609375
                        ):
    """
    Wstawia graczy na mapę w losowych area.
    Zapisuje nowy obraz oraz plik JSON z koordynatami.
    """

    result_img = map_img.copy()

    # Zmień rozmiar postaci
  

    coords_list = []

        
    area = random.choice(spawn_areas)
    for _ in range(num_players):
        coord = random_coord_in_area_with_size(area)
        x, y = coord
        
        x = int(x * scale)
        y = int(y * scale)

        coords_list.append({"x": x, "y": y})
        char_img = create_random_crewmate_return(sprites_hats, sprites_eyes, sprites_pants, sprites_pc)
        char_img_resized = cv2.resize(char_img, (char_width, char_height), interpolation=cv2.INTER_AREA)
        # Wklej gracza na mapę
        if char_img_resized.shape[2] == 4:
            # Kanał alfa
            alpha_s = char_img_resized[:, :, 3] / 255.0
            alpha_l = 1.0 - alpha_s

            for c in range(3):
                result_img[y:y+char_height, x:x+char_width, c] = (
                    alpha_s * char_img_resized[:, :, c] +
                    alpha_l * result_img[y:y+char_height, x:x+char_width, c]
                )
        else:
            result_img[y:y+char_height, x:x+char_width] = char_img_resized

            
    # Koordynaty gracza
    player_coords = {'x': x, 'y': y}

    # Rozdzielczość monitora
    screen_size = (2560, 1440)

    img = result_img
    img_h, img_w = img.shape[:2]
    screen_w, screen_h = screen_size
    center_x, center_y = player_coords['x'], player_coords['y']

    # Oblicz granice cropa
    left = max(center_x - screen_w // 2, 0)
    top = max(center_y - screen_h // 2, 0)
    right = left + screen_w
    bottom = top + screen_h

    if right > img_w:
        right = img_w
        left = max(right - screen_w, 0)

    if bottom > img_h:
        bottom = img_h
        top = max(bottom - screen_h, 0)

    # Przytnij
    cropped = img[top:bottom, left:right]

    # Zapisz
    cv2.imwrite(output_image_path, cropped)
  #  print(coords_list)

    return coords_list



In [225]:
from openpyxl import Workbook

def save_coords_to_xlsx(data_list, xlsx_path):
    """
    Zapisuje listę (map_name, list of points) do pliku XLSX.

    Args:
        data_list (list): Lista krotek (map_name, [{'x': x, 'y': y}, ...])
        xlsx_path (str): Ścieżka do pliku XLSX.

    Returns:
        None
    """
    wb = Workbook()
    ws = wb.active
    ws.title = "PlayerCoords"

    # Nagłówki
    ws.append(['map_name', 'point_index', 'x', 'y'])

    for map_name, points in data_list:
        for idx, point in enumerate(points):
            ws.append([map_name, idx, point['x'], point['y']])

    wb.save(xlsx_path)
    print(f"✅ Zapisano dane do {xlsx_path}")


In [233]:
areas = load_json_file("D:\\Project Sherlock\\spawn_areas.json")
# Ścieżki
map_path = "D:\\Project Sherlock\\Grafiki\\Grafiki total\\Maps\\The_Skeld_map.jpg"
map_img = cv2.imread(map_path)

sprites_pc = os.listdir("D:\\Project Sherlock\\Grafiki\\Grafiki total\\PC")[1:]
sprites_eyes = os.listdir("D:\\Project Sherlock\\Grafiki\\Grafiki total\\Eyes")[1:]
sprites_pants = os.listdir("D:\\Project Sherlock\\Grafiki\\Grafiki total\\Pants")[1:]
sprites_hats = os.listdir("D:\\Project Sherlock\\Grafiki\\Grafiki total\\Hats")[1:]

out_image_coordinates = "D:\\Project Sherlock\\Grafiki\\Grafiki total\\Generated_maps\\Coordinates_list_per_map.csv"
save_path = "D:\\Project Sherlock\\Grafiki\\Grafiki total"

coordinates_list = []
    
for x in range(200):

    out_image_path = "D:\\Project Sherlock\\Grafiki\\Grafiki total\\Generated_maps\\Map_{}.png".format(str(x))
    
    coordinates = spawn_players_on_map_instant(
    
                    map_img=map_img,
                    sprites_hats=sprites_hats, 
                    sprites_eyes=sprites_eyes, 
                    sprites_pants=sprites_pants, 
                    sprites_pc=sprites_pc,

                    spawn_areas=areas,
                    output_image_path=out_image_path,
                    num_players=random.randint(1,4)    
                                                    )
    
    coordinates_list.append(("Map_{}.png".format(str(x)), coordinates))
   # print("Created Map_{}.png".format(str(x)))

save_coords_to_xlsx(coordinates_list, "D:\\Project Sherlock\\Grafiki\\Grafiki total\\Generated_maps\\maps_points.xlsx")

✅ Wczytano 25 areas z D:\Project Sherlock\spawn_areas.json
✅ Zapisano dane do D:\Project Sherlock\Grafiki\Grafiki total\Generated_maps\maps_points.xlsx
